In [89]:
import numpy as np
import anytree as at
import pandas as pd
from graphviz import render
from anytree.dotexport import RenderTreeGraph, DotExporter


class DTreeRegressor(object):

    def __init__(self, min_sample_split=1, max_depth=999):
        self.msp = min_sample_split
        self.md = max_depth
        self.root = None

    def fit(self, data):
        root_split = self.get_split(data)
        root_node = self.create_node(root_split)
        self.grow(root_split, root_node, self.md, self.msp, 1)
        self.root = root_node
        print(at.RenderTree(root_node))

    # for a dataset, find best feature and value for split:
    def get_split(self, data, debug=False):

        if isinstance(data, pd.core.frame.DataFrame):
            df = True
            features = data.columns
            data = data.to_numpy()
        else:
            df = False

        best_score = np.infty
        best_split = None
        root = None
        for i in np.arange(data.shape[1] - 1):
            feature_i = data[:, i]
            split = self.choose_split(data, feature_i)
            curr_score = split['score']
            
            print(f'feature {i}')
            print(f'feature score: {split["score"]}')
            print(f'best score: {best_score}')
            
            if curr_score <= best_score:
                best_split = split
                best_score = curr_score
                feature = i
            print(f'new best score: {best_split["score"]}')
            print('------------------------')

        if df:
            best_split['feature'] = features[feature]
        else:
            best_split['feature'] = feature

        if debug:
            print(f'get_root debug')
            print(split)
            print('-------------------')

        return best_split

    # for a given feature, find best split of data. data sorted by feature argsort
    def choose_split(self, data, feature, debug=False):

        sort_idx = feature.argsort()
        feature = feature[sort_idx]
        data = data[sort_idx]
        target = data[:, -1]

        best_score = np.infty
        best_groups = None
        best_index = None

        for i in np.arange(len(feature) - 1):
            score = self.score_split(i, target)
            if score <= best_score:
                best_score = score
                best_index = i
                best_groups = self.get_groups(i, data)

        if debug:
            print(debug)
            print('Choose Split Debug:')
            print(f'feature # {feature}')
            print(f'best score is {best_score}')
            print(f'split at feature value {feature[best_index]}')
            print('End Debug: ----------------')

        return {'score': best_score,
                'samples': len(data),
                'target_avg': target.mean(),
                'mse': self.mse(target.mean(), target),
                'split_val': feature[best_index],
                'groups': best_groups}

    # create an AnyTree Node object from split data
    def create_node(self, split_info, parent=None):

        feature = split_info['feature']
        thresh = split_info['split_val']
        route = f'feature {feature} <= {thresh}'
        label = f'{route}\n' + \
                f'samples: {split_info["samples"]}\n' + \
                f'value: {split_info["target_avg"]}\n' + \
                f'mse: {split_info["mse"]}'
        return at.AnyNode(parent=parent, label=label)
        # 'route': route,
        # 'samples': split_info['samples'],
        # 'value': split_info['target_avg'],
        # 'mse': split_info['mse']
        # )

    def grow(self, split, node, max_depth, min_sample_split, depth):
        
        print('-------ENTER GROW-----------')

        left, right = split['groups']
        del split['groups']

        # check for a no split
        if (left.size == False) or (right.size == False):
            print('------------NO SPLIT-----------------')
            split['left'] = split['right'] = self.terminate(left + right)
            left_node = at.AnyNode(split['left'], parent=node)
            right_node = at.AnyNode(split['right'], parent=node)
            return

        # check max depth
        if depth >= max_depth:
            print('-------------MAX DEPTH EXCEEDED---------------')
            split['left'], split['right'] = self.terminate(left), self.terminate(right)
            left_node = at.AnyNode(split['left'], parent=node)
            right_node = at.AnyNode(split['right'], parent=node)
            return

        # start working on left node
        if len(left) <= min_sample_split:
            print('--------------LEFT NODE MIN SAMPLE----------------')
            split['left'] = self.terminate(left)
            left_node = at.AnyNode(parent=node, **split['left'])
        else:
            print('------------ SPLITTING LEFT NODE----------------')
            split['left'] = self.get_split(left)
            left_node = self.create_node(split['left'], parent=node)
            self.grow(split['left'], left_node, max_depth, min_sample_split, depth + 1)

        # start working on right node
        if len(right) <= min_sample_split:
            print('--------------RIGHT NODE MIN SAMPLE----------------')
            split['right'] = self.terminate(left)
            right_node = at.AnyNode(parent=node, **split['right'])
        else:
            print('------------ SPLITTING RIGHT NODE----------------')
            split['right'] = self.get_split(right)
            right_node = self.create_node(split['right'], parent=node)
            self.grow(split['right'], right_node, max_depth, min_sample_split, depth + 1)

    def terminate(self, group):
        target = group[:, -1]
        avg_val = target.mean()
        label = f'value: {avg_val}\n' \
                + f'mse: {self.mse(avg_val, target)}\n' \
                + f'samples: {len(group)}\n'

        outcome = {'label': label}

        return outcome

    # get left and right target means
    def get_means(self, y1, y2):
        return y1.mean(), y2.mean()

    # compute mean squared error
    def mse(self, mean, y):
        return (1 / len(y)) * np.sum((y - mean) ** 2)

    # get groups based on last index of left split
    def get_groups(self, index, data):
        left, right = np.array([]), np.array([])
        left = data[:(index + 1)]
        right = data[(index + 1):]
        return left, right

    # score a split based on weighted error (weight are sample sizes of splits)
    def score_split(self, index, target):
        ly, ry = self.get_groups(index, target)
        lm, rm = ly.mean(), ry.mean()
        return len(ly) * self.mse(lm, ly) + len(ry) * self.mse(rm, ry)

    def to_dot(self):
        self.node_namer()
        DotExporter(self.root, nodeattrfunc=lambda n: f'label="{n.label}"').to_dotfile('tree.dot')
        render('dot', 'png', 'tree.dot')

    def node_namer(self):
        i = 0
        for node in at.PreOrderIter(self.root):
            node.name = f'node_{i}'
            i += 1

In [90]:
test = pd.DataFrame({'height': [62, 34, 62, 55],
                     'weight': [1, 5, 9, 12],
                     'grade' : [7, 3, 4, 19],
                     'iq'    : [123, 555, 800, 2],
                     'Age'   : [5, 11, 15, 18]})
test

,height,weight,grade,iq,Age
0,62,1,7,123,5
1,34,5,3,555,11
2,62,9,4,800,15
3,55,12,19,2,18


In [91]:
dtree = DTreeRegressor()
dtree.fit(test)
dtree.to_dot()

feature 0
feature score: 74.5
best score: inf
new best score: 74.5
------------------------
feature 1
feature score: 22.5
best score: 74.5
new best score: 22.5
------------------------
feature 2
feature score: 50.66666666666666
best score: 22.5
new best score: 22.5
------------------------
feature 3
feature score: 50.66666666666666
best score: 22.5
new best score: 22.5
------------------------
-------ENTER GROW-----------
------------ SPLITTING LEFT NODE----------------
feature 0
feature score: 0.0
best score: inf
new best score: 0.0
------------------------
feature 1
feature score: 0.0
best score: 0.0
new best score: 0.0
------------------------
feature 2
feature score: 0.0
best score: 0.0
new best score: 0.0
------------------------
feature 3
feature score: 0.0
best score: 0.0
new best score: 0.0
------------------------
-------ENTER GROW-----------
--------------LEFT NODE MIN SAMPLE----------------
--------------RIGHT NODE MIN SAMPLE----------------
------------ SPLITTING RIGHT NODE